In [27]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import tqdm as tdqm
import matplotlib.pyplot as plt
from statsbombpy import sb
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression



In [ ]:
comps = sb.competitions()
comps[['competition_id', 'competition_name', 'season_id','season_name']].head(20)
# Womens Super League 2018/19
matches = sb.matches(competition_id = 37, season_id = 90)
matches[['match_id','home_team','away_team','match_date']].head()
# Pick a match and get the events
match_id = int(matches.iloc[0]['match_id'])
raw_events = sb.events(match_id=match_id)

ev = raw_events.copy()

def _xyz_from_list(series):
    """ Turn a column of [x,y] or [x, y,z] lists (or None) into two float cloumns."""
    def pad_xyz(v):
        if isinstance(v, (list, tuple)):
            padded = list(v) + [np.nan] * (3 - len(v))
            return padded[:3]
        return [np.nan, np.nan, np.nan]
    tmp = series.apply(pad_xyz)
    return pd.DataFrame(tmp.tolist(), index=series.index, columns=['x', 'y','z'])
# start coords (all event types)
ev[['x', 'y', 'z']] = _xyz_from_list(ev.get('location', pd.Series(index=ev.index)))
# end coords initiated as NaN
ev['end_x'] = np.nan
ev['end_y'] = np.nan
ev['end_z'] = np.nan
# fill end coords by event type
def fill_end(ev, type_name, end_colname):
    m = ev['type'].eq(type_name) & ev[end_colname].notna()
    if m.any():
        end_xyz = _xyz_from_list(ev.loc[m, end_colname])
        ev.loc[m, ['end_x', 'end_y', 'end_z']] = end_xyz.values

#passes, carries, shots  (add others later if needed)
fill_end(ev,'Pass','pass_end_location')
fill_end(ev,'Carry','carry_end_location')
fill_end(ev,'Shot','shot_end_location')

# Check
ev[['type', 'x', 'y', 'end_x', 'end_y']].head(12)
#raw_events[['team', 'player', 'type', 'x', 'y', 'end_x', 'end_y']].head(15)

In [ ]:
# Target Pitch dimensions (in meters)
PITCH_L, PITCH_W = 105.0, 68.0
#Statsbomb native pitch dimensions (in meters)
SB_L, SB_W = 120.0, 80.0

events = ev.copy()
# Keep only relevant columns
for c in ['team', 'player', 'type' ]